In [145]:
import pandas as pd
import pymssql
from clickhouse_driver import Client
from sqlalchemy import create_engine
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [146]:
pd. set_option('display.max_rows', 200)

In [147]:
def execute_clickhouse(query="select 'Test' as Test",
                       database='mart_hr',
                       external_tables=None,
                       **kwargs) -> pd.DataFrame:
    """
    Функция для выполнения запросов к Clickhouse

    **Аргументы:**

    - `query: str` - запрос
    - `database: str` - база данных (по-умолчанию `'dns_log'`)
    - `external_tables: pandas.DataFrame` - датафрэйм из которого бдует генерироваться внешняя таблица

    **Возвращает:**
    >    `pandas.DataFrame` с результатом запроса
    """
    client = Client(
        host='',
        port='',
        database=database,
        user='',
        password=''
    )
    if external_tables:
        pass
        #external_tables = [df_to_table(table[0], table[1]) for table in external_tables]
    rv = client.execute(query, external_tables=external_tables, with_column_types=True)
    df = pd.DataFrame(rv[0])
    df.columns = [i[0] for i in rv[1]]
    return df

def execute_pg_hr(query):
    """Функция обращения к хранилищу mart_hr DNS на чтение
    Параметры:
    query - строка, SQL запрос
    Возвращает - pandas DataFrame с содержанием результата выполнения запроса, названия столбцов из запроса"""

    host = ''
    schema = ''
    login = ''
    password = ''

    conection_url = f'postgresql+psycopg2://{login}:{password}@{host}/{schema}'

    engine = create_engine(conection_url)
    df = pd.read_sql(query, engine)
    engine.dispose()
    return df

def get_postgres_connection_string(connection_id) -> str:
    """
    Возвращает строку подключения к postgres для указанного подключения
    :return:
    """
    class connection:
        login = ''
        password = ''
        host = ''
        port = ''
        schema = ''

    return f'postgresql+psycopg2://{connection.login}:{connection.password}@{connection.host}:{connection.port}/{connection.schema}'

def execute_onec(query):
    """Функция обращения к Хранилищу DNS на чтение
    Параметры:
    query - строка, SQL запрос
    Возвращает - pandas DataFrame с содержанием результата выполнения запроса, названия столбцов из запроса"""
    ADuser = ''
    ADpassword = ''
    # ADuser = Config.ONEC_LOGIN
    # ADpassword = Config.ONEC_PASSWORD
    conn = pymssql.connect(server='', user=ADuser, password=ADpassword, database='')
    df = pd.read_sql(query, conn)
    
    return df



In [148]:
def GetPosLvlSalary():
    query = f"""                                                                                                                                                                               
	 WITH first_query AS
            (
            SELECT  
            --		reg_table.Должность AS Должность
                    ШтатноеРасписание.Наименование AS Должность
                    ,reg_table.Уровень AS Уровень
                    ,dwh.[Справочник.Города].Наименование AS Город
                    ,dwh.[Справочник.Города].Код AS КодГорода
                    ,reg_table.Значение AS Зарплата
                    , MAX(Период) OVER (PARTITION BY reg_table.Город, reg_table.Должность, reg_table.Уровень) AS max_period
                    ,reg_table.Период as Период
                    ,ШтатноеРасписание.ЛидерскийУровень  
            FROM dwh.[РегистрСведений.УровеньЗаработнойПлаты] AS reg_table
            JOIN dwh.[Справочник.Города] ON reg_table.Город = dwh.[Справочник.Города].Ссылка 
            JOIN dwh.[Справочник.ШтатноеРасписание] ШтатноеРасписание ON reg_table.Должность  =  ШтатноеРасписание.Ссылка  
                AND ШтатноеРасписание.ПрефиксКДолжности NOT IN  ('ДОМ', 'ГРП')
            )

            SELECT Город,КодГорода, Должность, Уровень, Зарплата, Период, max_period, CONVERT(DATE, GETDATE()) AS ДатаОбновления, ЛидерскийУровень
            FROM  first_query
            WHERE Период = max_period
                AND Должность != '2.03. Фед.развитие сайта'
                AND Должность not like '%(на аутсорсе)%' 
            ORDER BY Должность, Уровень
			 """
    df = execute_onec(query)
    return df

In [149]:
def getActualStaff():
    query = f"""
  select
     as2."Должность"
    , as2."УровеньОплаты"
    , as2."ТекущийФилиал"
    ,count(*) as КоличествоСотрудников
    , as2."Город"
    , as2."Филиал"
  from actual_staff as2
  where as2."Должность" not like '%%(на аутсорсе)%%'
  group by as2."Должность", as2."УровеньОплаты", as2."ТекущийФилиал", as2."ДатаОбновления", as2."Город", as2."Филиал"
  order by as2."Должность", as2."ТекущийФилиал", as2."УровеньОплаты" 
  """
    df = execute_pg_hr(query)
    return df

In [150]:
def getBranchCity():
	query = f"""
	select 
	  bh.BranchCode 
	, bh.BranchName as "Филиал"
	, replace(bh.CityName, 'РКЦ ', '') as "Город"
from dns_retail.BranchHier bh 
	"""
	df = execute_clickhouse(query)
	return df


In [151]:
def getLossBranches():

	"""НазваниеФирмы	НазваниеГорода	Должность"""

	query = f"""
	WITH МаксДатаЦТЕ AS (
	select 
	--	DATEFROMPARTS(YEAR(Дата), MONTH(Дата), 1) AS Дата
			Дата
		, сф.Наименование AS НазваниеФирмы 
		, сг.Код AS КодГорода 
		, сг.Наименование as НазваниеГорода
		, сф.Код as Фирма
		, сшр.Наименование as Должность
		, душрс.КоличествоСтавок
		, max(Дата) OVER (PARTITION BY сф.Код, сшр.Наименование) AS МаксДата
	from dwh.[Документ.УтверждениеШтатногоРасписания] душр
	join (
		select Фирма, YEAR(Дата) as Год, MONTH(Дата) as Месяц, MAX(DAY(Дата)) as День
		from dwh.[Документ.УтверждениеШтатногоРасписания] душрa 
		group by Фирма, YEAR(Дата), MONTH(Дата)
		) tmp
		on tmp.Год = YEAR(душр.Дата)
		and tmp.Месяц = MONTH(душр.Дата)
		and tmp.День = DAY(душр.Дата)
		and tmp.Фирма = душр.Фирма 
	join dwh.[Документ.УтверждениеШтатногоРасписания.Состав] душрс
		on душрс.Ссылка = душр.Ссылка
	join dwh.[Справочник.Фирмы] сф 
		on сф.Ссылка = душр.Фирма 
	join dwh.[Справочник.ШтатноеРасписание] сшр 
		on сшр.Ссылка = душрс.Должность 
	JOIN dwh.[Справочник.Города] сг 
		ON сф.Город = сг.Ссылка 
	where Дата >= '2021-01-01'
		and Проведен = 0x01
	--	    AND сф.Код = 1293
	--	order by душр.Дата

	), result_table AS
	(
	SELECT НазваниеФирмы, НазваниеГорода, Должность
	FROM МаксДатаЦТЕ
	WHERE МаксДата = Дата
	group by НазваниеФирмы, НазваниеГорода, Должность
	)

	select rt1.*
		, 1 as Уровень
	from result_table rt1
	union all 
	select rt2.*
		, 2 as Уровень
	from result_table rt2
	union all
	select rt3.*
		, 3 as Уровень
	from result_table rt3				

	"""
	df = execute_onec(query)
	return df

In [152]:
df_PosLvlSalary = GetPosLvlSalary()
df_PosLvlSalary

,Город,КодГорода,Должность,Уровень,Зарплата,Период,max_period,ДатаОбновления,ЛидерскийУровень
0,Владивосток,1.0,Web-дизайнер,1.0,50000.0,2020-02-01,2020-02-01,2022-07-19,1.0
1,Владивосток,1.0,Web-дизайнер,2.0,55000.0,2020-02-01,2020-02-01,2022-07-19,1.0
2,Владивосток,1.0,Web-дизайнер,3.0,60000.0,2020-02-01,2020-02-01,2022-07-19,1.0
3,Владивосток,1.0,Администратор киностудии,1.0,70000.0,2022-01-01,2022-01-01,2022-07-19,1.0
4,Владивосток,1.0,Администратор киностудии,2.0,80000.0,2022-01-01,2022-01-01,2022-07-19,1.0
...,...,...,...,...,...,...,...,...,...
63719,Владивосток,1.0,Юрист в отдел по работе с поставщиками,2.0,60000.0,2019-02-01,2019-02-01,2022-07-19,1.0
63720,Владивосток,1.0,Юрист в отдел по работе с поставщиками,3.0,65000.0,2019-02-01,2019-02-01,2022-07-19,1.0
63721,Владивосток,1.0,Юрист службы сервиса,1.0,70000.0,2022-07-01,2022-07-01,2022-07-19,1.0
63722,Владивосток,1.0,Юрист службы сервиса,2.0,80000.0,2022-07-01,2022-07-01,2022-07-19,1.0


In [153]:
df_actual_staff = getActualStaff()
df_actual_staff

,Должность,УровеньОплаты,ТекущийФилиал,КоличествоСотрудников,Город,Филиал
0,Администратор киностудии,2.0,2421.0,1,Владивосток,Видеостудия Владивосток
1,Администратор офиса,2.0,284.0,1,Москва,Адм. дивизиона Центральный
2,Администратор офиса,3.0,292.0,1,Воронеж,Адм. дивизиона Черноземье
3,Администратор офиса,1.0,897.0,1,Новосибирск,Адм. дивизиона Западная Сибирь
4,Администратор офиса,2.0,897.0,1,Новосибирск,Адм. дивизиона Западная Сибирь
...,...,...,...,...,...,...
18570,Юрист,3.0,5822.0,1,Владивосток,Направление ЮО по работе с поставщиками
18571,Юрист,1.0,5823.0,2,Владивосток,Направление ЮО по работе с розницей
18572,Юрист,2.0,5823.0,1,Владивосток,Направление ЮО по работе с розницей
18573,Юрист,3.0,5823.0,1,Владивосток,Направление ЮО по работе с розницей


In [154]:
df_loss_branches = getLossBranches()
df_loss_branches

,НазваниеФирмы,НазваниеГорода,Должность,Уровень
0,Федеральная дирекция,РКЦ Владивосток,Директор,1
1,Федеральная дирекция,РКЦ Владивосток,Директор по закупкам,1
2,Федеральная дирекция,РКЦ Владивосток,Руководитель службы обучения и развития персонала,1
3,Федеральная дирекция,РКЦ Владивосток,Федеральный бизнес аналитик,1
4,Федеральная дирекция,РКЦ Владивосток,Федеральный директор,1
...,...,...,...,...
59554,Яшкино в Центре,Яшкино,Зам. управляющего магазина,3
59555,Яшкино в Центре,Яшкино,Универсальный сотрудник магазина,3
59556,Яшкино в Центре,Яшкино,Универсальный сотрудник магазина 1 категории,3
59557,Яшкино в Центре,Яшкино,Универсальный сотрудник магазина 2 категории,3


In [155]:
df_main =  df_loss_branches\
    .merge(df_actual_staff, left_on=['НазваниеФирмы', 'Должность', 'Уровень'], right_on=['Филиал', 'Должность', 'УровеньОплаты'], how='left')
df_main['Город'] = df_main['Город'].fillna(df_main['НазваниеГорода'])
df_main['Филиал'] = df_main['Филиал'].fillna(df_main['НазваниеФирмы'])
df_main['КоличествоСотрудников'] = df_main['КоличествоСотрудников'].fillna(0)
df_main

,НазваниеФирмы,НазваниеГорода,Должность,Уровень,УровеньОплаты,ТекущийФилиал,КоличествоСотрудников,Город,Филиал
0,Федеральная дирекция,РКЦ Владивосток,Директор,1,NaN,NaN,0.0,РКЦ Владивосток,Федеральная дирекция
1,Федеральная дирекция,РКЦ Владивосток,Директор по закупкам,1,NaN,NaN,0.0,РКЦ Владивосток,Федеральная дирекция
2,Федеральная дирекция,РКЦ Владивосток,Руководитель службы обучения и развития персонала,1,NaN,NaN,0.0,РКЦ Владивосток,Федеральная дирекция
3,Федеральная дирекция,РКЦ Владивосток,Федеральный бизнес аналитик,1,NaN,NaN,0.0,РКЦ Владивосток,Федеральная дирекция
4,Федеральная дирекция,РКЦ Владивосток,Федеральный директор,1,NaN,NaN,0.0,РКЦ Владивосток,Федеральная дирекция
...,...,...,...,...,...,...,...,...,...
59710,Яшкино в Центре,Яшкино,Зам. управляющего магазина,3,NaN,NaN,0.0,Яшкино,Яшкино в Центре
59711,Яшкино в Центре,Яшкино,Универсальный сотрудник магазина,3,NaN,NaN,0.0,Яшкино,Яшкино в Центре
59712,Яшкино в Центре,Яшкино,Универсальный сотрудник магазина 1 категории,3,3.0,3784.0,1.0,Яшкино,Яшкино в Центре
59713,Яшкино в Центре,Яшкино,Универсальный сотрудник магазина 2 категории,3,NaN,NaN,0.0,Яшкино,Яшкино в Центре


In [156]:
df_checkerboard_salary = df_PosLvlSalary.merge(df_main, left_on=['Город','Должность', 'Уровень'], right_on=['Город', 'Должность', 'Уровень'], how='left')
df_checkerboard_salary = df_checkerboard_salary[['Город', 'Филиал', 'Должность', 
                        'Уровень', 'Зарплата','КоличествоСотрудников']]

In [157]:
df_checkerboard_salary[df_checkerboard_salary['Филиал'] == 'Владивосток Галактика КБТ']

,Город,Филиал,Должность,Уровень,Зарплата,КоличествоСотрудников
3878,Владивосток,Владивосток Галактика КБТ,Зам. управляющего магазина,1.0,60500.0,0.0
7309,Владивосток,Владивосток Галактика КБТ,Зам. управляющего магазина,2.0,66000.0,1.0
9116,Владивосток,Владивосток Галактика КБТ,Зам. управляющего магазина,3.0,71500.0,0.0
19008,Владивосток,Владивосток Галактика КБТ,Кладовщик магазина,1.0,41000.0,0.0
21552,Владивосток,Владивосток Галактика КБТ,Кладовщик магазина,2.0,45000.0,1.0
22998,Владивосток,Владивосток Галактика КБТ,Кладовщик магазина,3.0,49500.0,1.0
25904,Владивосток,Владивосток Галактика КБТ,Консультант дискаунтера,1.0,33500.0,0.0
26035,Владивосток,Владивосток Галактика КБТ,Консультант дискаунтера,2.0,40500.0,0.0
26090,Владивосток,Владивосток Галактика КБТ,Консультант дискаунтера,3.0,43000.0,0.0
33565,Владивосток,Владивосток Галактика КБТ,Продавец - консультант,1.0,33500.0,0.0


In [158]:
df_checkerboard_salary.isna().sum() # 3898

Город                        0
Филиал                   35701
Должность                    0
Уровень                      0
Зарплата                     0
КоличествоСотрудников    35701
dtype: int64

In [159]:
filt_test1 = df_test_1['УровеньОплаты'].isna()
df_test_1[filt_test1]

,НазваниеФирмы,НазваниеГорода,Должность,Уровень,УровеньОплаты,ТекущийФилиал,КоличествоСотрудников,Город,Филиал
0,Федеральная дирекция,РКЦ Владивосток,Директор,1,NaN,NaN,0.0,РКЦ Владивосток,Федеральная дирекция
1,Федеральная дирекция,РКЦ Владивосток,Директор по закупкам,1,NaN,NaN,0.0,РКЦ Владивосток,Федеральная дирекция
2,Федеральная дирекция,РКЦ Владивосток,Руководитель службы обучения и развития персонала,1,NaN,NaN,0.0,РКЦ Владивосток,Федеральная дирекция
3,Федеральная дирекция,РКЦ Владивосток,Федеральный бизнес аналитик,1,NaN,NaN,0.0,РКЦ Владивосток,Федеральная дирекция
4,Федеральная дирекция,РКЦ Владивосток,Федеральный директор,1,NaN,NaN,0.0,РКЦ Владивосток,Федеральная дирекция
...,...,...,...,...,...,...,...,...,...
59709,яЧишмы ТЦ Дружба (ЗАКРЫТО),Чишмы,Управляющий маг. средней категории,3,NaN,NaN,0.0,Чишмы,яЧишмы ТЦ Дружба (ЗАКРЫТО)
59710,Яшкино в Центре,Яшкино,Зам. управляющего магазина,3,NaN,NaN,0.0,Яшкино,Яшкино в Центре
59711,Яшкино в Центре,Яшкино,Универсальный сотрудник магазина,3,NaN,NaN,0.0,Яшкино,Яшкино в Центре
59713,Яшкино в Центре,Яшкино,Универсальный сотрудник магазина 2 категории,3,NaN,NaN,0.0,Яшкино,Яшкино в Центре


In [160]:
# filt = (df_checkerboard_salary['НазваниеФирмы'] == 'Владивосток Галактика КБТ') & (df_checkerboard_salary['Филиал'] == 'Владивосток Галактика КБТ') 
filt1 = (df_test_1['НазваниеФирмы'] == 'Владивосток Галактика КБТ')
df_test_1[filt1].head(135).sort_values(by=['Должность', 'Уровень'])



,НазваниеФирмы,НазваниеГорода,Должность,Уровень,УровеньОплаты,ТекущийФилиал,КоличествоСотрудников,Город,Филиал
3194,Владивосток Галактика КБТ,Владивосток,Зам. управляющего магазина,1,NaN,NaN,0.0,Владивосток,Владивосток Галактика КБТ
23131,Владивосток Галактика КБТ,Владивосток,Зам. управляющего магазина,2,2.0,1703.0,1.0,Владивосток,Владивосток Галактика КБТ
43023,Владивосток Галактика КБТ,Владивосток,Зам. управляющего магазина,3,NaN,NaN,0.0,Владивосток,Владивосток Галактика КБТ
3195,Владивосток Галактика КБТ,Владивосток,Кладовщик магазина,1,NaN,NaN,0.0,Владивосток,Владивосток Галактика КБТ
23132,Владивосток Галактика КБТ,Владивосток,Кладовщик магазина,2,2.0,1703.0,1.0,Владивосток,Владивосток Галактика КБТ
43024,Владивосток Галактика КБТ,Владивосток,Кладовщик магазина,3,3.0,1703.0,1.0,Владивосток,Владивосток Галактика КБТ
3196,Владивосток Галактика КБТ,Владивосток,Консультант дискаунтера,1,NaN,NaN,0.0,Владивосток,Владивосток Галактика КБТ
23133,Владивосток Галактика КБТ,Владивосток,Консультант дискаунтера,2,NaN,NaN,0.0,Владивосток,Владивосток Галактика КБТ
43025,Владивосток Галактика КБТ,Владивосток,Консультант дискаунтера,3,NaN,NaN,0.0,Владивосток,Владивосток Галактика КБТ
3197,Владивосток Галактика КБТ,Владивосток,Продавец - консультант,1,NaN,NaN,0.0,Владивосток,Владивосток Галактика КБТ


In [161]:
df_checkerboard_salary['Филиал'].isna().sum()

35701

In [163]:
# df_checkerboard_salary['НазваниеФирмы'].isna().sum()

In [ ]:
df_checkerboard_salary['Филиал'].isna().sum()

0

In [ ]:
df_checkerboard_salary

,Город,КодГорода,Должность,Уровень,Зарплата,Период,max_period,ДатаОбновления,ЛидерскийУровень,НазваниеФирмы,НазваниеГорода,УровеньОплаты,ТекущийФилиал,КоличествоСотрудников,Филиал
0,Новосибирск,19.0,Администратор офиса,1.0,30000.0,2019-02-01,2019-02-01,2022-07-19,1.0,Адм. дивизиона Западная Сибирь,Новосибирск,1.0,897.0,1.0,Адм. дивизиона Западная Сибирь
1,Новосибирск,19.0,Администратор офиса,2.0,35000.0,2021-04-01,2021-04-01,2022-07-19,1.0,Адм. дивизиона Западная Сибирь,Новосибирск,2.0,897.0,1.0,Адм. дивизиона Западная Сибирь
2,Новосибирск,19.0,Администратор офиса,3.0,40000.0,2021-04-01,2021-04-01,2022-07-19,1.0,Адм. дивизиона Западная Сибирь,Новосибирск,NaN,NaN,0.0,Адм. дивизиона Западная Сибирь
3,Воронеж,59.0,Администратор офиса,1.0,25000.0,2021-11-01,2021-11-01,2022-07-19,1.0,Адм. дивизиона Черноземье,Воронеж,1.0,5836.0,1.0,Отдел персонала Черноземье
4,Воронеж,59.0,Администратор офиса,1.0,25000.0,2021-11-01,2021-11-01,2022-07-19,1.0,Отдел персонала Черноземье,Воронеж,1.0,5836.0,1.0,Отдел персонала Черноземье
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214928,Екатеринбург,24.0,Юрисконсульт,2.0,60000.0,2021-12-01,2021-12-01,2022-07-19,1.0,Юридический Отдел (д. Урал),Екатеринбург,2.0,5379.0,1.0,Юридический Отдел (д. Урал)
214929,Екатеринбург,24.0,Юрисконсульт,3.0,70000.0,2021-12-01,2021-12-01,2022-07-19,1.0,Юридический Отдел (д. Урал),Екатеринбург,NaN,NaN,0.0,Юридический Отдел (д. Урал)
214930,Владивосток,1.0,Юрисконсульт,1.0,60000.0,2021-12-01,2021-12-01,2022-07-19,1.0,Адм. дивизиона Дальний Восток,Владивосток,NaN,NaN,0.0,Адм. дивизиона Дальний Восток
214931,Владивосток,1.0,Юрисконсульт,2.0,70000.0,2021-12-01,2021-12-01,2022-07-19,1.0,Адм. дивизиона Дальний Восток,Владивосток,2.0,899.0,1.0,Адм. дивизиона Дальний Восток


In [ ]:
df_checkerboard_salary['Филиал'] = df_checkerboard_salary['Филиал'].fillna(df_checkerboard_salary['НазваниеФирмы'])

In [ ]:
df = df_checkerboard_salary[['Город','Должность', 'Уровень', 'Зарплата', 'Филиал', 'ЛидерскийУровень', 'КоличествоСотрудников']]

In [ ]:
qqq = df_checkerboard_salary.groupby(['Город','Должность', 'Уровень', 'Зарплата', 'Филиал', 'ЛидерскийУровень', 'КоличествоСотрудников'], as_index=False)['Город','Должность', 'Уровень', 'Зарплата', 'Филиал', 'ЛидерскийУровень', 'КоличествоСотрудников']

C:\Users\peredereev.ro\AppData\Local\Temp\ipykernel_14816\461515408.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  qqq = df_checkerboard_salary.groupby(['Город','Должность', 'Уровень', 'Зарплата', 'Филиал', 'ЛидерскийУровень', 'КоличествоСотрудников'], as_index=False)['Город','Должность', 'Уровень', 'Зарплата', 'Филиал', 'ЛидерскийУровень', 'КоличествоСотрудников']


In [ ]:
qqq.size()

,Город,Должность,Уровень,Зарплата,Филиал,ЛидерскийУровень,КоличествоСотрудников,size
0,Абаза,Универсальный сотрудник магазина,1.0,31000.0,Абаза ТК Околица,1.0,1.0,1
1,Абаза,Универсальный сотрудник магазина,2.0,34000.0,Абаза ТК Околица,1.0,3.0,1
2,Абаза,Универсальный сотрудник магазина,3.0,37000.0,Абаза ТК Околица,1.0,0.0,1
3,Абаза,Управляющий маг.,1.0,55000.0,Абаза ТК Околица,2.0,0.0,1
4,Абаза,Управляющий маг.,2.0,60000.0,Абаза ТК Околица,2.0,1.0,1
...,...,...,...,...,...,...,...,...
41602,Яшкино,Универсальный сотрудник магазина 2 категории,2.0,25400.0,Яшкино в Центре,1.0,1.0,1
41603,Яшкино,Универсальный сотрудник магазина 2 категории,3.0,28000.0,Яшкино в Центре,1.0,0.0,1
41604,Яшкино,Управляющий маг. средней категории,1.0,65000.0,Яшкино в Центре,2.0,0.0,1
41605,Яшкино,Управляющий маг. средней категории,2.0,80000.0,Яшкино в Центре,2.0,1.0,1


In [ ]:
df_checkerboard_salary[df_checkerboard_salary['Зарплата'].isna()]['Город'].unique()


array([], dtype=object)

In [ ]:
# filt = df_checkerboard_salary['Город'] == 'Владивосток' 
# df_checkerboard_salary.loc[filt][df_checkerboard_salary['Зарплата'].isna()]
# qqq = qqq[qqq['Филиал'].isna()]
# qqq